In [ ]:
#  !pip install transformers seqeval[gpu]

In [ ]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 37.0 MB/s 
     |████████████████████████████████| 880 kB 42.2 MB/s 
     |████████████████████████████████| 3.0 MB 43.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e30848dd497c0325a0c55845890dfa7a3a3686a6776048ac5d39df0a1f1e9917
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
from pprint import pprint
from google.colab import drive
from google.colab import files
import regex as re
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from transformers import RobertaTokenizerFast, RobertaConfig, RobertaForTokenClassification
import torch
from torch.utils.data import Dataset, DataLoader
from torch import cuda
from sklearn.metrics import accuracy_score

import glob
import os.path
import numpy as np
import sys
import codecs

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
train_folder = "/content/drive/MyDrive/NLP/project_5_data/datasets/train-articles" 
dev_folder = "/content/drive/MyDrive/NLP/project_5_data/datasets/dev-articles"    
train_labels_file = "/content/drive/MyDrive/NLP/project_5_data/datasets/train-labels-task-si/"
dev_labels_file = "/content/drive/MyDrive/NLP/project_5_data/datasets/dev-labels-task-si"
task_TC_output_file = "/content/drive/MyDrive/NLP/project_5_data/baseline-output-TC.txt"

In [ ]:
def read_articles_from_file_list(folder_name, file_pattern="*.txt"):
    """
    Read articles from files matching patterns <file_pattern> from  
    the directory <folder_name>. 
    The content of the article is saved in the dictionary whose key
    is the id of the article (extracted from the file name).
    Each element of <sentence_list> is one line of the article.
    """
    file_list = glob.glob(os.path.join(folder_name, file_pattern))
    articles = {}
    article_id_list, sentence_id_list, sentence_list = ([], [], [])
    for filename in sorted(file_list):
        article_id = os.path.basename(filename).split(".")[0][7:]
        with codecs.open(filename, "r", encoding="utf8") as f:
            articles[article_id] = f.read()
    return articles

In [ ]:
dev_folder = "/content/drive/MyDrive/NLP/project_5_data/datasets/dev-articles" # check that the path to the datasets folder is correct, if not adjust these variables accordingly 
propaganda_techniques_file = "/content/drive/MyDrive/NLP/project_5_data/propaganda-techniques-scorer/data/propaganda-techniques-names-semeval2020task11.txt" # propaganda_techniques_file is in the tools.tgz file (download it from the team page)
task_SI_output_file = "/content/drive/MyDrive/NLP/project_5_data/bert_base_SI_output.txt"

In [ ]:
# !python3 task-SI_scorer.py -s project_5_data/baseline-output-SI.txt -r project_5_data/datasets/dev-labels-task-si/

In [ ]:
file_list = glob.glob(os.path.join(train_folder, "*.txt"))
train_articles_content, train_articles_id = ([], [])
for filename in file_list:
    with open(filename, "r", encoding="utf-8") as f:
        train_articles_content.append(' '.join([line.strip() for line in f]))
        train_articles_id.append(os.path.basename(filename).split(".")[0][7:])


In [ ]:
train_articles = read_articles_from_file_list(train_folder)
dev_articles = read_articles_from_file_list(dev_folder)

len(train_articles), len(dev_articles)

(371, 75)

In [ ]:
TASK_3_ARTICLE_ID_COL = 0
#TASK_3_TECHNIQUE_NAME_COL = 1
TASK_3_FRAGMENT_START_COL = 1
TASK_3_FRAGMENT_END_COL = 2

def extract_article_id_from_file_name(fullpathfilename):

    regex = re.compile("article([0-9]+).*")
    return regex.match(os.path.basename(fullpathfilename)).group(1)

   
def load_annotation_list_from_folder(folder_name, techniques_names):

    file_list = glob.glob(os.path.join(folder_name, "*.labels"))
    if len(file_list)==0:
        print("Cannot load file list in folder " + folder_name)
        sys.exit()
    annotations = {}
    for filename in file_list:
        annotations[extract_article_id_from_file_name(filename)] = []
        with open(filename, "r") as f:
            for row_number, line in enumerate(f.readlines()):
                row = line.rstrip().split("\t")
                annotations[row[TASK_3_ARTICLE_ID_COL]].append((row[TASK_3_FRAGMENT_START_COL], row[TASK_3_FRAGMENT_END_COL]))

    return annotations

In [ ]:
techniques_names = [ "propaganda" ]
train_annotation = load_annotation_list_from_folder(train_labels_file, techniques_names)

In [ ]:
train_labels = {}
for article in train_articles.keys():
    labels = [0] * len(train_articles[article])
    for annot in train_annotation[article]:
        labels[int(annot[0]):int(annot[1])+1] = [1] * (int(annot[1]) - int(annot[0]) + 1)
    train_labels[article] = labels

In [ ]:
train_labels_str = {}
for article_id in train_articles.keys():
    index = 0
    word_index = 0
    # labels = [0] * len(train_articles[article_id].replace('\n\n',' ').split(' '))
    labels = [0] * len(train_articles[article_id].replace('\n\n',' ').replace('\n', ' ').strip().split(' '))
    labels_str = ['O'] * len(labels)

    for sentence in train_articles[article_id].replace('\n', ' ').strip().split('  '):
        for word in sentence.split(' '):
            if train_labels[article_id][index] == 1:
                labels[word_index] = 1
                labels_str[word_index] = 'I-Prop'
            word_index += 1
            index += len(word) + 1
        index += 1

    train_labels[article_id] = labels
    train_labels_str[article_id] = labels_str

In [ ]:
len(train_labels), len(train_labels_str)

(371, 371)

In [ ]:
train_df = pd.DataFrame(columns = ['ID','text','labels'])
count = 0
for article_id in train_articles.keys():
    train_df.loc[count,'ID'] = article_id
    train_df.loc[count,'text'] = train_articles[article_id].replace('\n\n',' ').replace('\n', ' ').strip()
    temp_label = [str(i) for i in train_labels_str[article_id]]
    train_df.loc[count,'labels'] = ','.join(temp_label)
    count+=1
# del train_df['ID']

In [ ]:
train_df

,ID,text,labels
0,111111111,Next plague outbreak in Madagascar could be 's...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,111111112,US bloggers banned from entering UK Two promin...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,111111113,Kate Steinle's death at the hands of a Mexican...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,111111114,U.S. judge frees Indonesian immigrant held by ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,111111115,Here are all the sexual misconduct accusations...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
...,...,...,...
366,999001296,Altered Election Documents Tied To Florida Dem...,"O,O,O,O,O,O,O,O,O,O,O,I-Prop,I-Prop,I-Prop,I-P..."
367,999001297,Migrant Caravan Reach Border & Climb Atop Fenc...,"O,O,O,O,O,I-Prop,I-Prop,I-Prop,I-Prop,O,O,O,O,..."
368,999001619,Guardian ups its vilification of Julian Assang...,"O,O,O,I-Prop,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
369,999001621,This Guardian Fake News Story Proves That The ...,"I-Prop,I-Prop,I-Prop,I-Prop,I-Prop,I-Prop,I-Pr..."


In [ ]:
new_train_df = pd.DataFrame(columns = train_df.columns)
count = 0
no_words = 256
for i in range(len(train_df)):
  start = 0
  while(True):
    text1 = ' '.join(train_df.loc[i,'text'].split(' ')[start:start+no_words])
    new_train_df.loc[count,'ID'] = train_df.loc[i,'ID']
    new_train_df.loc[count,'text'] = text1
    new_train_df.loc[count,'labels'] = ','.join(train_df.loc[i,'labels'].split(',')[start:start+no_words])
    count+=1
    start+=no_words
    if(start>=len(train_df.loc[i,'text'].split(' '))):
      break

In [ ]:
new_train_df

,ID,text,labels
0,111111111,Next plague outbreak in Madagascar could be 's...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,111111111,not over. The larger-than-usual outbreak had h...,"I-Prop,I-Prop,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,111111112,US bloggers banned from entering UK Two promin...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,111111112,"speakers who promote hate."" EDL leader Tommy R...","I-Prop,I-Prop,I-Prop,I-Prop,O,O,O,O,O,O,O,O,O,..."
4,111111112,"home secretary believes they would ""continue t...","O,O,O,O,O,O,O,O,O,O"
...,...,...,...
1522,999001621,"it. Jonathan Cook, a former Guardian writer, m...","I-Prop,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
1523,999001621,the journalists? ... Harding is likely a major...,"I-Prop,I-Prop,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1524,999001621,| Permalink Comments,"O,O,O"
1525,999001970,"SNL Indian Comedian Silenced for ""Offensive Jo...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [ ]:
len(new_train_df.loc[2,'text'].split(' ')), len(new_train_df.loc[2,'labels'].split(','))

(256, 256)

In [ ]:
train_df = new_train_df
del train_df['ID']

In [ ]:
# labels_to_ids = {k: v for v, k in enumerate(['B-Prop','I-Prop', 'O'])}
# ids_to_labels = {v: k for v, k in enumerate(['B-Prop','I-Prop', 'O'])}
# labels_to_ids, ids_to_labels

labels_to_ids = {k: v for v, k in enumerate(['I-Prop', 'O'])}
ids_to_labels = {v: k for v, k in enumerate(['I-Prop', 'O'])}
labels_to_ids, ids_to_labels

({'I-Prop': 0, 'O': 1}, {0: 'I-Prop', 1: 'O'})

In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 1
DEV_BATCH_SIZE = 1
EPOCHS = 7
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip().split(' ')  
        word_labels = self.data.labels[index].split(",") 


        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        encoding = self.tokenizer(sentence,
                             is_pretokenized=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 

        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [ ]:
train_size = 1
train_dataset = train_df.sample(frac=train_size,random_state=200)
test_dataset = train_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(train_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1527, 2)
TRAIN Dataset: (1527, 2)
TEST Dataset: (0, 2)


In [ ]:
training_set[1209]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1

In [ ]:
print(tokenizer.decode(training_set[1209]["input_ids"].tolist()))

[CLS] “ ‘ Or, Mike, take the firearms first, and then go to court, ’ Trump responded. ” See the column here : Florida School Massacre Proves Police Are Worthless In Protecting Us Well, Trump ’ s statements are now the rallying cry for gun grabbers all over America to enact gun confiscation laws. So far, five states have passed “ red flag ” laws that allow police agencies to confiscate guns from someone deemed to be “ dangerous ” by either a law enforcement officer or a family member BEFORE the individual has committed any crime, and in one State ( Rhode Island ) the governor issued an executive order implementing a “ red flag ” law. Here are the six states where the legislatures have already passed — or the governor has already issued an executive order implementing — “ red flag ” laws : California Connecticut Indiana Oregon Washington Rhode Island And since Trump ’ s Stalinist statements, there are at least 24 additional states that are currently considering passing “ red flag ” laws.

In [ ]:
for token, mapping, label in zip(tokenizer.convert_ids_to_tokens(training_set[1209]["input_ids"]), training_set[1209]['offset_mapping'], training_set[1209]["labels"]):
    print('{0:10} {1:4}  {2:15}'.format(token, label.item(), str(mapping.tolist())))
    

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
# testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(labels_to_ids))
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss, outputs

(tensor(0.6762, device='cuda:0', grad_fn=<NllLossBackward0>),
 (tensor(0.6762, device='cuda:0', grad_fn=<NllLossBackward0>),
  tensor([[[-0.1447, -0.1800],
           [ 0.0464, -0.0893],
           [-0.1756,  0.0261],
           ...,
           [-0.1696, -0.0362],
           [-0.2404, -0.0022],
           [-0.2694,  0.0035]]], device='cuda:0', grad_fn=<AddBackward0>)))

In [ ]:
input_ids.shape

torch.Size([1, 512])

In [ ]:
labels.shape

torch.Size([1, 512])

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 2])

In [ ]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(7):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.6731213331222534
Training loss per 100 training steps: 0.36469194492196094
Training loss per 100 training steps: 0.36604528881572373
Training loss per 100 training steps: 0.35950693683667834
Training loss per 100 training steps: 0.35241291623683346
Training loss per 100 training steps: 0.3470288377158537
Training loss per 100 training steps: 0.34486925150089576
Training loss per 100 training steps: 0.3387239161246428
Training loss epoch: 0.33997073834420655
Training accuracy epoch: 0.8688673192917337
Training epoch: 2
Training loss per 100 training steps: 0.35091111063957214
Training loss per 100 training steps: 0.26659484679746154
Training loss per 100 training steps: 0.27978445767466703
Training loss per 100 training steps: 0.29411227562065634
Training loss per 100 training steps: 0.2948704770048546
Training loss per 100 training steps: 0.29211264815434307
Training loss per 100 training steps: 0.2895309706815145
Training loss 

In [ ]:
torch.save(model, "/content/drive/MyDrive/NLP/BERT_Task_SI.pt")
# model = torch.load("/content/drive/MyDrive/NLP/BERT_Task_SI.pt", map_location=torch.device('cpu'))
# model.eval()

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            # temp_labels = labels
            # print(idx,labels)
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            # print(temp_labels == labels)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            # eval_labels.extend(flattened_targets)
            # eval_preds.extend(flattened_predictions)
            
            # tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
# labels, predictions = valid(model, testing_loader)

In [ ]:
dev_annotation = load_annotation_list_from_folder(dev_labels_file, techniques_names)

dev_labels = {}
for article in dev_articles.keys():
    labels = [0] * len(dev_articles[article])
    for annot in dev_annotation[article]:
        labels[int(annot[0]):int(annot[1])] = [1] * (int(annot[1]) - int(annot[0]))
    dev_labels[article] = labels


In [ ]:
dev_labels_str = {}
for article_id in dev_articles.keys():
    index = 0
    word_index = 0

    labels = [0] * len(dev_articles[article_id].replace('\n\n','\n').replace('\n', ' ').strip().split(' '))
    labels_str = ['O'] * len(labels)
    
    first_sentence = True
    for sentence in dev_articles[article_id].replace('\n\n', '\n').strip().split('\n'):
        for word in sentence.split(' '):
            if dev_labels[article_id][index] == 1:
                labels[word_index] = 1
                labels_str[word_index] = 'I-Prop'
            word_index += 1
            index += len(word) + 1
        if first_sentence:
            first_sentence = False
            index += 1
        # index += 1

    dev_labels[article_id] = labels
    dev_labels_str[article_id] = labels_str


In [ ]:
dev_df = pd.DataFrame(columns = ['ID','text','labels'])
count = 0
for article_id in dev_articles.keys():
    dev_df.loc[count,'ID'] = article_id
    dev_df.loc[count,'text'] = dev_articles[article_id].replace('\n\n','\n').replace('\n', ' ').strip()
    temp_label = [label for label in dev_labels_str[article_id]]
    dev_df.loc[count,'labels'] = ','.join(temp_label)
    count+=1


In [ ]:
new_dev_df = pd.DataFrame(columns = dev_df.columns)
count = 0
no_words = 256
for i in range(len(dev_df)):
    start = 0
    while(True):
        text1 = ' '.join(dev_df.loc[i,'text'].split(' ')[start:start+no_words])
        new_dev_df.loc[count,'ID'] = dev_df.loc[i,'ID']
        new_dev_df.loc[count,'text'] = text1
        new_dev_df.loc[count,'labels'] = ','.join(dev_df.loc[i,'labels'].split(',')[start:start+no_words])
        count+=1
        start+=no_words
        if(start>=len(dev_df.loc[i,'text'].split(' '))):
            break

dev_df = new_dev_df
dev_id = dev_df['ID']
del dev_df['ID']

In [ ]:
dev_df

,text,labels
0,Police had previously gone to home where Ohio ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,"could be heard yelling, ‘‘We have shots fired....","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,"America's Immigration Voice. Earlier, I blogge...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,I-Pr..."
3,"since 2017, some for alleged domestic violence...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,Man arrested for allegedly buying gun used in ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
...,...,...
257,Press and the Los Angeles Times are among the ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
258,Trump Vows to Kick CNN’s Acosta Out Again Pres...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
259,"news conference,"" Trump said. Acosta’s hard pa...","I-Prop,I-Prop,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
260,forward to continuing to defend the White Hous...,"O,O,O,O,O,O,O,O,I-Prop,O,O,I-Prop,I-Prop,O,O,O..."


In [ ]:
#All data in dev set
dev_dataset = dev_df

dev_set = dataset(dev_dataset, tokenizer, MAX_LEN)
dev_params = {'batch_size': DEV_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }


dev_loader = DataLoader(dev_set, **dev_params)

In [ ]:
dev_id

0      730081389
1      730081389
2      730093263
3      730093263
4      730246508
         ...    
257    999001323
258    999001419
259    999001419
260    999001419
261    999001419
Name: ID, Length: 262, dtype: object

In [ ]:
# Run 1 by 1
prediction_map = {}
actual_labels_map = {}

for i in range(len(dev_df)):
    single_dev_row = pd.DataFrame(columns = ['text','labels'])
    single_dev_row.loc[0, 'text'] = dev_df.loc[i].text
    single_dev_row.loc[0, 'labels'] = dev_df.loc[i].labels

    dev_set = dataset(single_dev_row, tokenizer, MAX_LEN)
    dev_loader = DataLoader(dev_set, **dev_params)
    labels, predictions = valid(model, dev_loader)
    if dev_id[i] not in prediction_map:
        prediction_map[dev_id[i]] = []
    prediction_map[dev_id[i]].append(predictions)
    if dev_id[i] not in actual_labels_map:
        actual_labels_map[dev_id[i]] = []
    actual_labels_map[dev_id[i]].append(labels)

Validation loss per 100 evaluation steps: 0.0010113143362104893
Validation Loss: 0.0010113143362104893
Validation Accuracy: 1.0
Validation loss per 100 evaluation steps: 0.003582330420613289
Validation Loss: 0.003582330420613289
Validation Accuracy: 1.0
Validation loss per 100 evaluation steps: 0.057770878076553345
Validation Loss: 0.057770878076553345
Validation Accuracy: 0.98828125
Validation loss per 100 evaluation steps: 0.6879169344902039
Validation Loss: 0.6879169344902039
Validation Accuracy: 0.8256880733944955
Validation loss per 100 evaluation steps: 0.0011655106209218502
Validation Loss: 0.0011655106209218502
Validation Accuracy: 1.0
Validation loss per 100 evaluation steps: 0.02388973906636238
Validation Loss: 0.02388973906636238
Validation Accuracy: 0.99609375
Validation loss per 100 evaluation steps: 0.30000999569892883
Validation Loss: 0.30000999569892883
Validation Accuracy: 0.89453125
Validation loss per 100 evaluation steps: 0.6449406147003174
Validation Loss: 0.644940

In [ ]:
len(prediction_map.keys())

75

In [ ]:
labels_actual, predictions = valid(model, dev_loader)

Validation loss per 100 evaluation steps: 0.02570517733693123
Validation Loss: 0.02570517733693123
Validation Accuracy: 0.991869918699187


In [ ]:
total_word_len = []
for article in dev_df.text:
    total_word_len.append(len(article.split(' ')))
np.mean(total_word_len), len(dev_df.text)

(220.54580152671755, 262)

In [ ]:
predictions.count('I-Prop'), predictions.count('B-Prop'), predictions.count('O'), len(predictions)

(3, 0, 120, 123)

In [ ]:
correct_word_prediction = 0
for index in range(len(labels)):
  if labels[index] == predictions[index] and labels[index] is not 'O':
    correct_word_prediction+=1
    print(index, labels[index])

print(correct_word_prediction)

46 I-Prop
47 I-Prop
2


In [ ]:
task_SI_output_file = '/content/drive/MyDrive/NLP/project_5_data/bert_base_SI_output_256tokens_e6.txt'

In [ ]:
len(prediction_map.keys())

75

In [ ]:
with open(task_SI_output_file, "w") as fout:
    
    for dev_article_id in dev_articles.keys():
        print("Processing:", dev_article_id)
        article_text = dev_articles[dev_article_id]
        predictions_list = prediction_map[dev_article_id]

        print("Word count in article:", len(article_text.replace('\n\n','\n').replace('\n', ' ').strip().split(' ')))

        # pred_labels = [0] * len(predictions)
        # for i in range(len(predictions)):
        #     if predictions[i] == 'I-Prop':
        #         pred_labels[i] = 1
        
        pred_labels = []
        for predictions in predictions_list:
            for temp_pred_label in predictions:
                if temp_pred_label == 'I-Prop':
                    pred_labels.append(1)
                else:
                    pred_labels.append(0)
        print("Prediction length for article {0:9}:{1:5}".format(dev_article_id, len(pred_labels)))

        char_index = 0
        span_started = False
        span_len = 0
        label_index = 0
        idx = 0

        while idx < len(article_text) and label_index < len(pred_labels):
        # for idx in range(len(article_text)):
            # print("idx: {0:5}, char: {1:2}, label_idx: {2:5}".format(idx, article_text[idx], label_index))

            if (article_text[idx:idx+2] == '\n\n') and span_started:
                idx += 2
                label_index += 1
                span_len += 2
                continue
            elif (article_text[idx:idx+2] == '\n\n'):
                idx += 2
                label_index +=1
                continue
            if (article_text[idx] == ' ' or article_text[idx] == '\n') and span_started:
                label_index += 1
                span_len += 1
                idx += 1
                continue
            elif article_text[idx] == ' ' or article_text[idx] == '\n':
                label_index += 1
                idx += 1
                continue
                        
            if pred_labels[label_index] == 1 and not span_started:
                span_started = True
                start = idx
                span_len += 1
            elif pred_labels[label_index] == 1 and span_started:
                span_len += 1
            elif pred_labels[label_index] == 0 and span_started:
                span_started = False          
                if span_len > 0:
                    fout.write("%s\t%s\t%s\n" % (dev_article_id, start, start+span_len))
                    span_len = 0
            idx += 1
        print("Done Processing\n")
    print("Predictions writted to file: ", task_SI_output_file)

In [ ]:
%cd /content/drive/MyDrive/NLP/
!python3 task-SI_scorer.py -s project_5_data/bert_base_SI_output_256tokens_e6.txt -r project_5_data/datasets/dev-labels-task-si/